In [5]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Preprocess function
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text

# Additional function to check for contradictions (manual rules)
def check_contradiction(post, incident_description):
    contradiction_phrases = [
        "dozens of people dead", "genocide", "russia never attacked", "no major damage"
    ]
    for phrase in contradiction_phrases:
        if phrase in post:
            return True
    return False

# Function to classify posts based on similarity and contradictions
def classify_posts(incident_description, social_media_posts, threshold=0.25):
    # Preprocess the incident description and posts
    incident_description = preprocess_text(incident_description)
    social_media_posts = [preprocess_text(post) for post in social_media_posts]
    
    # Combine the incident description and posts for vectorization
    all_text = [incident_description] + social_media_posts
    
    # Convert to TF-IDF vector representation
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(all_text)
    
    # Calculate cosine similarity between incident description and each post
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    
    # Classify posts as true or false based on the threshold and contradictions
    true_posts = []
    false_posts = []
    
    for idx, score in enumerate(similarity_scores):
        post = social_media_posts[idx]
        
        # Check if the post has contradictions, if so classify as false
        if check_contradiction(post, incident_description):
            false_posts.append(post)
        # Otherwise, classify based on similarity threshold
        elif score >= threshold:
            true_posts.append(post)
        else:
            false_posts.append(post)
    
    return true_posts, false_posts

# Example usage
incident_description = input("Describe what really happened: ")

# Get multiple social media posts from the user
print("\nEnter social media posts related to the incident. Type 'done' when finished:")
social_media_posts = []
while True:
    post = input(f"Post {len(social_media_posts) + 1}: ")
    if post.lower() == 'done':
        break
    social_media_posts.append(post)

# Classify the posts
true_posts, false_posts = classify_posts(incident_description, social_media_posts)

# Output the results
print("\nTrue Posts:")
for post in true_posts:
    print(f"- {post}")

print("\nFalse Posts:")
for post in false_posts:
    print(f"- {post}")


Describe what really happened:  A missile struck a residential building in Kyiv, causing significant damage but no fatalities. Ukrainian officials claim it was a Russian missile attack. Russia denies responsibility, claiming it was Ukrainian defense systems that caused the damage.



Enter social media posts related to the incident. Type 'done' when finished:


Post 1:  Russia just bombed Kyiv, dozens of people dead!
Post 2:  There was a missile strike in Kyiv, but it looks like it was the Ukrainian defense system causing the explosion.
Post 3:  Russia never attacked Kyiv. The explosion was caused by Ukrainian defense.
Post 4:  Ukrainian officials say the missile that hit Kyiv came from Russia, but there were no fatalities.
Post 5:  Russia is deliberately attacking civilians in Kyiv. This is an act of genocide!
Post 6:   Ukrainian defense intercepted Russian missiles, no major damage reported in Kyiv.
Post 7:  There are reports of explosions in Kyiv, but no confirmation of Russian involvement.
Post 8:  done



True Posts:
- there was a missile strike in kyiv but it looks like it was the ukrainian defense system causing the explosion
- ukrainian officials say the missile that hit kyiv came from russia but there were no fatalities

False Posts:
- russia just bombed kyiv dozens of people dead
- russia never attacked kyiv the explosion was caused by ukrainian defense
- russia is deliberately attacking civilians in kyiv this is an act of genocide
-  ukrainian defense intercepted russian missiles no major damage reported in kyiv
- there are reports of explosions in kyiv but no confirmation of russian involvement
